exe time of 
- eigenmapping


# Import some pkgs

In [7]:
import sys
sys.path.append("../../../mypkg")

import scipy
import itertools

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import trange
from scipy.io import loadmat
from functools import partial
from easydict import EasyDict as edict
from collections import defaultdict as ddict
from IPython.display import display
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# SBI and torch
from sbi.inference.base import infer
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi import analysis
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as sutils

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

In [9]:
# my own fns
from brain import Brain
from FC_utils import build_fc_freq_m
from constants import RES_ROOT, DATA_ROOT, FIG_ROOT
from utils.misc import load_pkl, save_pkl
from utils.reparam import theta_raw_2out, logistic_np, logistic_torch
from utils.measures import geodesic_dist, reg_R_fn, lin_R_fn, lap_mat_fn

plt.style.use(FIG_ROOT/"base.mplstyle")

# Some fns

In [10]:
_minmax_vec = lambda x: (x-np.min(x))/(np.max(x)-np.min(x));
_remove_norm = lambda x: x/np.linalg.norm(x)
_std_vec = lambda x: (x-np.mean(x))/np.std(x)
_remove_scale = _minmax_vec
uptri_idxs = np.triu_indices(68, k=1);
# transfer vec to a sym mat
def _vec_2mat(vec):
    mat = np.zeros((68, 68))
    mat[np.triu_indices(68, k = 1)] = vec
    mat = mat + mat.T
    return mat

# Load data and params

In [11]:
paras = edict()
paras.bds = ["delta", "theta", "alpha", "beta_l"]
#paras.bds = ["alpha"]
paras.allbds = ["delta", "theta", "alpha", "beta_l"]
paras.add_v = 0.01
paras.nepoch = 100

In [12]:
# em FC
fc_root = RES_ROOT/"emp_fcs2"
def _get_fc(sub_ix, bd):
    fil = list(fc_root.rglob(f"*{bd}*{paras.nepoch}/sub{sub_ix}.pkl"))[0]
    return np.abs(load_pkl(fil, verbose=False))

In [13]:

def _add_v2con(cur_ind_conn, add_v):
    cur_ind_conn = cur_ind_conn.copy()
    add_v = np.quantile(cur_ind_conn, 0.99)*add_v # tuning 0.1
    np.fill_diagonal(cur_ind_conn[:34, 34:68], np.diag(cur_ind_conn[:34, 34:68]) + add_v)
    np.fill_diagonal(cur_ind_conn[34:68, :34], np.diag(cur_ind_conn[34:68, :34]) + add_v)
    np.fill_diagonal(cur_ind_conn[68:77, 77:], np.diag(cur_ind_conn[68:77, 77:]) + add_v)
    np.fill_diagonal(cur_ind_conn[77:, 68:77], np.diag(cur_ind_conn[77:, 68:77]) + add_v)
    return cur_ind_conn


In [14]:
# SC
ind_conn_xr = xr.open_dataarray(DATA_ROOT/'individual_connectomes_reordered.nc')
ind_conn = ind_conn_xr.values;
#scs = ind_conn.transpose(2, 0, 1)[:, :68, :68];
#scs = np.array([_preprocess_sc(sc) for sc in scs]);

scs = []
for cur_ind_idx in range(36):
    # create spectrome brain:
    brain = Brain.Brain()
    brain.add_connectome(DATA_ROOT) # grabs distance matrix
    # re-ordering for DK atlas and normalizing the connectomes:
    brain.reorder_connectome(brain.connectome, brain.distance_matrix)
     # re-assign connectome to individual connectome
    brain.connectome =  _add_v2con(ind_conn[:, :, cur_ind_idx], paras.add_v)
    brain.bi_symmetric_c()
    brain.reduce_extreme_dir()
    sc = brain.reducedConnectome
    scs.append(sc[:68, :68])
scs = np.array(scs)
scs.shape

(36, 68, 68)

# Eigenmapping

In [15]:
from scipy.optimize import minimize
class EigMapping():
    """Do eigenmapping approx from SC to FC.
       lamnew = exp(-lam(SC)*p1)+p2 from ben
    """
    def __init__(self, sc, init_ps, rescale_fn=lambda x: x):
        """args:
            sc: the SC matrix
            init_ps: Initial guess of the parameters, a list of two
            rescale_fn: The fn for rescale fc, default is the identity, minmax does not converge.
        """
        
        self.init_ps = init_ps
        #sc = np.diag(1/np.sum(sc, axis=0)) @ sc
        self.eigvals, self.eigvecs = self._get_eigres(sc)
        self.rescale_fn = rescale_fn
        self.opt_ps = None
         
    def _get_eigres(self, sc):
        eigvals, eigvecs = np.linalg.eig(sc);
        sort_idxs = np.argsort(-np.abs(eigvals))
        eigvals, eigvecs = eigvals[sort_idxs], eigvecs[:, sort_idxs]
        return eigvals, eigvecs
        
    def _recon_fc(self, ps=None):
        p1, p2 = ps
        new_eigvals = np.exp(-p1*self.eigvals/np.median(self.eigvals))+p2
        # note that for an ordinary matrix, it should be Ulam U^-1
        #rec_fc = self.eigvecs @ np.diag(new_eigvals) @ np.linalg.inv(self.eigvecs)
        rec_fc = self.eigvecs @ np.diag(new_eigvals) @ self.eigvecs.T
        return rec_fc - np.diag(np.diag(rec_fc))
    
    def opt(self, fc):
        def fun(ps):
            newfc = self._recon_fc(ps)
            idxs = np.triu_indices_from(newfc, k=1)
            return -lin_R_fn(self.rescale_fn(fc[idxs]), self.rescale_fn(newfc[idxs]))[0]
            #return np.mean((self.rescale_fn(fc[idxs])-
            #                self.rescale_fn(newfc[idxs]))**2)
        res = minimize(fun, self.init_ps, bounds= ((0, 1000), (-1000, 1000)), 
                      options=dict(disp=False))
        self.opt_ps = res.x
        if not res.success:
            print(f"May not converge, {res.message}.")
            
    def get_optfc(self, fc):
        if self.opt_ps is None:
            self.opt(fc)
        return self._recon_fc(self.opt_ps)

In [17]:
import time 
band = "alpha"
ts = [time.time()]
for sub_ix in trange(10):
    opt = EigMapping(scs[sub_ix], [0, 0]) 
    curfc = _get_fc(sub_ix, band)
    opt.get_optfc(curfc);
    ts.append(time.time())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.37it/s]


In [18]:
dlt_ts = np.diff(ts)
print(f"The time for each subject is {np.mean(dlt_ts):.3f} with std {np.std(dlt_ts):.3f}.")

The time for each subject is 0.070 with std 0.041.
